# EndSARS Twitter Analysis

Sentiment and Network Analysis of the #EndSARS protest movement that occured in 2020 in Nigeria. 

## SetUp

### Import Python Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import yaml
import os
import json
import re
import ast
import csv
import gensim
import searchtweets
import fasttext as ft
from sklearn.model_selection import train_test_split
import numpy as np

%matplotlib inline

### Connecting to Twitter

I suggest you use Twitter's official searchtweets api which is easy to use and reliable.
<br>In order to get past the tweets retrieval limit, you might also need to  use the premium search api.

- Create a hidden yaml file (.yaml) which contains details about your twitter api keys
- Create a variable that stores the twitter api credentials from the hiddem yaml file above.

In [ ]:
from searchtweets import ResultStream, gen_request_parameters, load_credentials

search_args = load_credentials(".twitter_keys.yml",
                                       yaml_key="search_tweets_v2_recent",
                                       env_overwrite=False)

query = gen_request_parameters("EndSARS",results_per_call=10,start_time="2021-03-05",
    tweet_fields=('geo,public_metrics,context_annotations,created_at,author_id,possibly_sensitive,referenced_tweets'),
    user_fields=('created_at,description,entities,id,location,public_metrics,username'),
    place_fields=('country,geo,id,name,place_type'),
    expansions=('author_id,entities.mentions.username,referenced_tweets.id,referenced_tweets.id.author_id'),
    stringify=True)

print(query)

In [ ]:
from searchtweets import collect_results

tweets = collect_results(query,
                         max_tweets=10,
                         result_stream_args=search_args) # change this if you need to

In [ ]:
[print(tweet, end='\n\n') for tweet in tweets[0:10]]

In [ ]:
tweets_df = pd.DataFrame(tweets)

In [ ]:
tweets_df[:4]

In [ ]:
tweets_df.to_pickle('./endsars_tweets.pkl')

In [304]:
from searchtweets import ResultStream, gen_request_parameters, load_credentials
from searchtweets import collect_results


def retrieve_tweets(search_words, start_date, end_date):
    
    search_args = load_credentials(".twitter_keys.yml",
                                       yaml_key="search_tweets_v2",
                                       env_overwrite=False)
    
    query = gen_request_parameters(search_words, results_per_call=100, from_date = start_date, to_date = end_date,
    tweet_fields=('author_id,context_annotations,created_at,entities,geo,id,public_metrics,possibly_sensitive,referenced_tweets,text'),
    user_fields=('id,location,name,public_metrics,username,verified'),
    place_fields=('country,full_name,geo,name,place_type'),
    expansions=('author_id,referenced_tweets.id,geo.place_id,referenced_tweets.id.author_id'))

    tweets = collect_results(query,
                         max_tweets=10000,
                         result_stream_args=search_args)
    
    return tweets

In [305]:
# Initialise these variables:
search_words = "#EndSARS OR #Soro Soke OR #Police Brutality OR #lekki toll gate OR #anti-robbery sqaud OR #lekki massacre OR #End bad governance OR #End swat OR #buhari is a bad boy"
start_date = "2020-10-01"
end_date = "2020-10-21"

# Call the function scraptweets
#retrieve_tweets(search_words, start_date, end_date)

In [306]:
def create_tweets_df(tweets):
    
    tweets_df = pd.DataFrame(tweets)
    tweets_df.to_pickle('./new_endsars_tweets(look_up).pkl')
    
    return tweets_df

In [ ]:
look_up_df = pd,read_pickel(./new_endsars_tweets(look_up).pkl)

In [307]:
# Initialise these variables:
tweets=retrieve_tweets(search_words, start_date, end_date)

# Call the function create tweets into dataframe
create_tweets_df(tweets)

TypeError: gen_request_parameters() got an unexpected keyword argument 'from_date'

In [ ]:
import requests
import json

def auth():
    return "AAAAAAAAAAAAAAAAAAAAANE2NAEAAAAAS7YKh9sTQnSUY8ERjYLRiASf7Gw%3DOphmA1rPK5Xtc7BQcLAL9GRRas6mTFvEq7kd5U18ydBBsXj5M6"

def create_url():
    # Specify the usernames that you want to lookup below
    # You can enter up to 100 comma-separated values.
    usernames = "usernames=TwitterDev,TwitterAPI"
    user_fields = "user.fields=description,created_at"
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
    return url

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [3]:
# Reading retrieved tweets from storage

df_1=pd.read_pickle('./1stnew_endsars_tweets.pkl')
df_2=pd.read_pickle('./2ndnew_endsars_tweets.pkl')
df_3=pd.read_pickle('./anew_endsars_tweets(A).pkl')
df_4=pd.read_pickle('./anew_endsars_tweets(B).pkl')
df_5=pd.read_pickle('./anew_endsars_tweets(C).pkl')
df_6=pd.read_pickle('./new_endsars_tweets(A).pkl')
df_7=pd.read_pickle('./new_endsars_tweets(B).pkl')
df_8=pd.read_pickle('./new_endsars_tweets(C).pkl')

In [4]:
# Merging the dataframe as a single dataset

frames = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8]
data = pd.concat(frames)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69420 entries, 0 to 10203
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   public_metrics       68000 non-null  object 
 1   entities             67878 non-null  object 
 2   text                 68000 non-null  object 
 3   referenced_tweets    63956 non-null  object 
 4   possibly_sensitive   68000 non-null  object 
 5   id                   68000 non-null  object 
 6   created_at           68000 non-null  object 
 7   author_id            68000 non-null  object 
 8   context_annotations  15462 non-null  object 
 9   geo                  194 non-null    object 
 10  users                710 non-null    object 
 11  tweets               710 non-null    object 
 12  places               146 non-null    object 
 13  newest_id            710 non-null    object 
 14  oldest_id            710 non-null    object 
 15  result_count         710 non-null   

In [13]:
data.head()

,public_metrics,entities,text,referenced_tweets,possibly_sensitive,id,created_at,author_id,context_annotations,geo,users,tweets,places,newest_id,oldest_id,result_count,next_token
0,"{'retweet_count': 1, 'reply_count': 0, 'like_c...","{'hashtags': [{'start': 0, 'end': 8, 'tag': 'E...",#Endsars\n#EndNaijaKillings \n#SecureNorth \n#...,"[{'type': 'quoted', 'id': '1333161983953276928'}]",False,1333199034958426113,2020-11-29T23:59:54.000Z,302632039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'retweet_count': 1, 'reply_count': 0, 'like_c...","{'hashtags': [{'start': 15, 'end': 23, 'tag': ...",Fim de Sars - #Endsars [EP.32]\nhttps://t.co/...,NaN,False,1333199021574393864,2020-11-29T23:59:51.000Z,827180991882076161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'retweet_count': 22, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 12, 'usernam...",RT @pitharoy: @DrOlufunmilayo “They” think we ...,"[{'type': 'retweeted', 'id': '1333014726561894...",False,1333198975650902017,2020-11-29T23:59:40.000Z,893366796321001472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'retweet_count': 0, 'reply_count': 1, 'like_c...","{'mentions': [{'start': 0, 'end': 16, 'usernam...",@surayyah__ahmad @ChiMarieconsult @dawisu I li...,"[{'type': 'replied_to', 'id': '133281962901884...",False,1333198967102967808,2020-11-29T23:59:38.000Z,1317152652027957248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'retweet_count': 3854, 'reply_count': 0, 'lik...","{'mentions': [{'start': 3, 'end': 14, 'usernam...",RT @Mochievous: So many people still in custod...,"[{'type': 'retweeted', 'id': '1333040880383528...",False,1333198966016643072,2020-11-29T23:59:37.000Z,1318556292689133568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
data2 = data.copy()

In [26]:
#Removing duplicates in Dataframe

ref_cols = ['created_at', 'author_id']
data = data.loc[~data[ref_cols].duplicated()]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59558 entries, 0 to 10201
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   public_metrics       59557 non-null  object 
 1   entities             59439 non-null  object 
 2   text                 59557 non-null  object 
 3   referenced_tweets    56135 non-null  object 
 4   possibly_sensitive   59557 non-null  object 
 5   id                   59557 non-null  object 
 6   created_at           59557 non-null  object 
 7   author_id            59557 non-null  object 
 8   context_annotations  13804 non-null  object 
 9   geo                  167 non-null    object 
 10  users                1 non-null      object 
 11  tweets               1 non-null      object 
 12  places               1 non-null      object 
 13  newest_id            0 non-null      object 
 14  oldest_id            0 non-null      object 
 15  result_count         0 non-null     

In [30]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69420 entries, 0 to 10203
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   public_metrics       68000 non-null  object 
 1   entities             67878 non-null  object 
 2   text                 68000 non-null  object 
 3   referenced_tweets    63956 non-null  object 
 4   possibly_sensitive   68000 non-null  object 
 5   id                   68000 non-null  object 
 6   created_at           68000 non-null  object 
 7   author_id            68000 non-null  object 
 8   context_annotations  15462 non-null  object 
 9   geo                  194 non-null    object 
 10  users                710 non-null    object 
 11  tweets               710 non-null    object 
 12  places               146 non-null    object 
 13  newest_id            710 non-null    object 
 14  oldest_id            710 non-null    object 
 15  result_count         710 non-null   

In [40]:
data2.loc[data2[ref_cols].duplicated()]

,public_metrics,entities,text,referenced_tweets,possibly_sensitive,id,created_at,author_id,context_annotations,geo,users,tweets,places,newest_id,oldest_id,result_count,next_token
99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1333199034958426113,1333196546716938245,98.0,b26v89c19zqg8o3foset68czzlye0rlk3eu6d3oeh388t
200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'username': 'officialvaler', 'verified': Fal...","[{'id': '1315159437489864704', 'possibly_sensi...",NaN,NaN,NaN,NaN,NaN
201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1333196516874448899,1333193979362488327,100.0,b26v89c19zqg8o3foset68czv2xyviyivhhckg07jyt8d
300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'username': 'peter_nwos', 'id': '921639589',...",[{'text': 'So many people still in custody fro...,NaN,NaN,NaN,NaN,NaN
301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1333193973150724096,1333191356785831936,98.0,b26v89c19zqg8o3foset68czs0ezvefv8d643jrkf7rlp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1333418925707710464,1333417846295171080,99.0,b26v89c19zqg8o3foset6eos5q3o4tcdgyhol8qs6up31
10147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': '1050019489880961026', 'location': 'Ni...","[{'entities': {'mentions': [{'start': 222, 'en...","[{'id': '00817d75276a95cf', 'name': 'Lagos', '...",NaN,NaN,NaN,NaN
10148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1333417842767704064,1333416673085444096,99.0,b26v89c19zqg8o3foset6eos471zh12cb6qz14ea9lg8t
10202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'public_metrics': {'followers_count': 72, 'f...","[{'entities': {'mentions': [{'start': 51, 'end...",NaN,NaN,NaN,NaN,NaN


In [45]:
data.isnull().sum()

public_metrics             1
entities                 119
text                       1
referenced_tweets       3423
possibly_sensitive         1
id                         1
created_at                 1
author_id                  1
context_annotations    45754
geo                    59391
users                  59557
tweets                 59557
places                 59557
newest_id              59558
oldest_id              59558
result_count           59558
next_token             59558
dtype: int64

In [46]:
data.isnull().sum()*100/len(data)

public_metrics           0.001679
entities                 0.199805
text                     0.001679
referenced_tweets        5.747339
possibly_sensitive       0.001679
id                       0.001679
created_at               0.001679
author_id                0.001679
context_annotations     76.822593
geo                     99.719601
users                   99.998321
tweets                  99.998321
places                  99.998321
newest_id              100.000000
oldest_id              100.000000
result_count           100.000000
next_token             100.000000
dtype: float64

In [55]:
data.drop(['context_annotations', 'geo', 'users', 'tweets', 'places', 'newest_id',
       'oldest_id', 'result_count', 'next_token'], axis=1)

,public_metrics,entities,text,referenced_tweets,possibly_sensitive,id,created_at,author_id
0,"{'retweet_count': 1, 'reply_count': 0, 'like_c...","{'hashtags': [{'start': 0, 'end': 8, 'tag': 'E...",#Endsars\n#EndNaijaKillings \n#SecureNorth \n#...,"[{'type': 'quoted', 'id': '1333161983953276928'}]",False,1333199034958426113,2020-11-29T23:59:54.000Z,302632039
1,"{'retweet_count': 1, 'reply_count': 0, 'like_c...","{'hashtags': [{'start': 15, 'end': 23, 'tag': ...",Fim de Sars - #Endsars [EP.32]\nhttps://t.co/...,NaN,False,1333199021574393864,2020-11-29T23:59:51.000Z,827180991882076161
2,"{'retweet_count': 22, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 12, 'usernam...",RT @pitharoy: @DrOlufunmilayo “They” think we ...,"[{'type': 'retweeted', 'id': '1333014726561894...",False,1333198975650902017,2020-11-29T23:59:40.000Z,893366796321001472
3,"{'retweet_count': 0, 'reply_count': 1, 'like_c...","{'mentions': [{'start': 0, 'end': 16, 'usernam...",@surayyah__ahmad @ChiMarieconsult @dawisu I li...,"[{'type': 'replied_to', 'id': '133281962901884...",False,1333198967102967808,2020-11-29T23:59:38.000Z,1317152652027957248
4,"{'retweet_count': 3854, 'reply_count': 0, 'lik...","{'mentions': [{'start': 3, 'end': 14, 'usernam...",RT @Mochievous: So many people still in custod...,"[{'type': 'retweeted', 'id': '1333040880383528...",False,1333198966016643072,2020-11-29T23:59:37.000Z,1318556292689133568
...,...,...,...,...,...,...,...,...
10197,"{'retweet_count': 64, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 18, 'usernam...",RT @Mazi_KokoMazta: 2021 hope u re hearing us ...,"[{'type': 'retweeted', 'id': '1333341572382003...",False,1333415978642911233,2020-11-30T14:21:57.000Z,1322676745267486725
10198,"{'retweet_count': 7, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 3, 'end': 15, 'usernam...",RT @dokubohilda: Regime change qwa? How? Pleas...,"[{'type': 'retweeted', 'id': '1333410821897134...",False,1333415971609055233,2020-11-30T14:21:55.000Z,4267222247
10199,"{'retweet_count': 37, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 16, 'usernam...",RT @_MrsMalaprop: You people seem to have forg...,"[{'type': 'retweeted', 'id': '1333351251971624...",False,1333415965305032705,2020-11-30T14:21:54.000Z,2900186977
10200,"{'retweet_count': 200, 'reply_count': 0, 'like...","{'mentions': [{'start': 3, 'end': 19, 'usernam...",RT @SaharaReporters: #LekkiTollGateShooting: S...,"[{'type': 'retweeted', 'id': '1333344859202129...",False,1333415932732051457,2020-11-30T14:21:46.000Z,921036472639217665


In [281]:
data.entities.iloc[3]

#a = data.public_metrics.iloc[

{'mentions': [{'start': 0, 'end': 16, 'username': 'surayyah__ahmad'},
  {'start': 17, 'end': 33, 'username': 'ChiMarieconsult'},
  {'start': 34, 'end': 41, 'username': 'dawisu'}],
 'hashtags': [{'start': 169, 'end': 177, 'tag': 'EndSARS'}],
 'annotations': [{'start': 137,
   'end': 147,
   'probability': 0.6521,
   'type': 'Organization',
   'normalized_text': 'Arewa Youth'},
  {'start': 310,
   'end': 314,
   'probability': 0.5328,
   'type': 'Place',
   'normalized_text': 'North'}],
 'urls': [{'start': 317,
   'end': 340,
   'url': 'https://t.co/meKGC7XmPC',
   'expanded_url': 'https://twitter.com/UnamakaEjiofor/status/1333198967102967808/photo/1',
   'display_url': 'pic.twitter.com/meKGC7XmPC'}]}

In [116]:
def return_feature(x, var):
    if type(x) == float:
        pass
    else:
        return x.get(var)

#data[var] = data.public_metrics.apply(return_feature, args=('var',))

In [117]:
for feat in ['reply_count', 'like_count', 'quote_count']:
    data[feat] = data.public_metrics.apply(return_feature, args=(feat,))

In [ ]:
data['retweet_count'] = data.public_metrics.apply(return_feature)

In [131]:
data.drop('public_metrics',axis=1, inplace=True)

KeyError: "['public_metrics'] not found in axis"

In [138]:
data.head()

,entities,text,referenced_tweets,possibly_sensitive,id,created_at,author_id,context_annotations,geo,users,tweets,places,newest_id,oldest_id,result_count,next_token,retweet_count,reply_count,like_count,quote_count
0,"{'hashtags': [{'start': 0, 'end': 8, 'tag': 'E...",#Endsars\n#EndNaijaKillings \n#SecureNorth \n#...,"[{'type': 'quoted', 'id': '1333161983953276928'}]",False,1333199034958426113,2020-11-29T23:59:54.000Z,302632039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0
1,"{'hashtags': [{'start': 15, 'end': 23, 'tag': ...",Fim de Sars - #Endsars [EP.32]\nhttps://t.co/...,NaN,False,1333199021574393864,2020-11-29T23:59:51.000Z,827180991882076161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0
2,"{'mentions': [{'start': 3, 'end': 12, 'usernam...",RT @pitharoy: @DrOlufunmilayo “They” think we ...,"[{'type': 'retweeted', 'id': '1333014726561894...",False,1333198975650902017,2020-11-29T23:59:40.000Z,893366796321001472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,0.0,0.0,0.0
3,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",@surayyah__ahmad @ChiMarieconsult @dawisu I li...,"[{'type': 'replied_to', 'id': '133281962901884...",False,1333198967102967808,2020-11-29T23:59:38.000Z,1317152652027957248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,2.0,0.0
4,"{'mentions': [{'start': 3, 'end': 14, 'usernam...",RT @Mochievous: So many people still in custod...,"[{'type': 'retweeted', 'id': '1333040880383528...",False,1333198966016643072,2020-11-29T23:59:37.000Z,1318556292689133568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3854.0,0.0,0.0,0.0


In [275]:
data.possibly_sensitive.iloc[0]

False

In [280]:
set(data.possibly_sensitive)

{False, nan, True}

In [262]:
m = data.referenced_tweets.iloc[100]
type(m), m

(list, [{'type': 'retweeted', 'id': '1332006218320269312'}])

In [272]:
m[0].get('type')

def get_retweet(x):
    if type(x) != float and x[0].get('type') == 'retweeted':
        return 1
    else:
        return 0

In [273]:
#get_retweet(m)
data.referenced_tweets.apply(get_retweet)

0        0
1        0
2        1
3        0
4        1
        ..
10197    1
10198    1
10199    1
10200    1
10201    0
Name: referenced_tweets, Length: 59558, dtype: int64

In [ ]:
data.entities.iloc[0]

In [165]:
b=data.entities.iloc[0]
b.get('hashtags')
tag = [elt.get('tag') for elt in b.get('hashtags')]

In [175]:
#tag

['Endsars',
 'EndNaijaKillings',
 'SecureNorth',
 'EndBadGovernanceinNIGERIA',
 'EndInsecurity',
 'EndOppression',
 'EnoughIsEnough']

In [166]:
#len(set(tag))
type({})

dict

In [189]:
def return_hashtags(x, var):
    
    if type(x) == None: 
        pass

    else:
        x.get(var)
        for elt in x.get('hashtags'):
            if type(elt) == None:
                pass
            else:
                tag = []
                tag_ = elt.get('tag')
                #print(tag_)
                tag.append(tag_)
        return tag

#data[var] = data.entities.apply(return_hashtags, args=('var',))

In [221]:
def return_hashtags(x):
    
    if type(x) == None or type(x) == float: 
        pass
    else:
        hashtag = x.get('hashtags')
        if hashtag == None:
            return []
        else:
            tag = [elt.get('tag') for elt in hashtag]
            return tag

In [219]:
return_hashtags(data.entities.iloc[0])
#data.drop('hashtags', axis=1, inplace=True)

In [222]:
data['hashtags'] = data.entities.apply(return_hashtags)

In [223]:
data.hashtags

0        [Endsars, EndNaijaKillings, SecureNorth, EndBa...
1                                                [Endsars]
2                                                [EndSARS]
3                                                [EndSARS]
4                                                [EndSARS]
                               ...                        
10197                              [BuhariResign, EndSARS]
10198                                                   []
10199                                                   []
10200                     [LekkiTollGateShooting, EndSARS]
10201                                [TachaIfeMi, EndSARS]
Name: hashtags, Length: 59558, dtype: object

In [134]:
data.text.iloc[10]

'RT @sars_watch: Teenagers still in police custody at Lion-building prison After being arrested during the #EndSARS protest \n\nhttps://t.co/9…'

In [114]:
a= data.public_metrics.iloc[0]
#return_feature(a)
a

{'retweet_count': 1, 'reply_count': 0, 'like_count': 1, 'quote_count': 0}

In [136]:
data.referenced_tweets.iloc[10]

[{'type': 'retweeted', 'id': '1332378126643589123'}]

In [111]:
data['retweet_count']

0           1.0
1           1.0
2          22.0
3           0.0
4        3854.0
          ...  
10197      64.0
10198       7.0
10199      37.0
10200     200.0
10201       0.0
Name: retweet_count, Length: 59558, dtype: float64

In [224]:
data.columns

Index(['entities', 'text', 'referenced_tweets', 'possibly_sensitive', 'id',
       'created_at', 'author_id', 'context_annotations', 'geo', 'users',
       'tweets', 'places', 'newest_id', 'oldest_id', 'result_count',
       'next_token', 'retweet_count', 'reply_count', 'like_count',
       'quote_count', 'hashtags'],
      dtype='object')

In [69]:
type(data.public_metrics)

pandas.core.series.Series

In [68]:
data.public_metrics.str.extract(r'[retweet_count](\d)', expand=False)

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
10197    NaN
10198    NaN
10199    NaN
10200    NaN
10201    NaN
Name: public_metrics, Length: 59558, dtype: object

## Fetch and Clean Data

Use the functions you've created above to collect all the tweets

dates_list = ["2020-10-01", "2020-10-08",
              "2020-10-15", "2020-10-22", "2020-10-29",
              "2020-11-05", "2020-11-12",
              "2020-11-19", "2020-11-26",
              "2017-11-30"]

tweets_df = create_tweets_df(dates_list)
tweets_df.shape

Make sure to save the tweets after this process, use a file format that won't comprimise the characters in tweets

In [ ]:
tweets_df.to_csv("./data/endsars_tweets.csv", index=False)

In [ ]:
def delete_duplicates(df):
    """
    Create a function that looks at the tweets in the tweets dataframe and other characters and delete 
    duplicates as sometimes we can get duplicate data/tweets. For example if I tweet something and someone else
    retweets it then this could possibly be a duplicate. Is this useful information? You need to decide of course.
    """

def preprocess_tweet(tweet):
    """
    Create a function that preprocess the text in a single tweet. Suggestions for this function are:
    - correct all multiple white spaces to a single white space
    - convert all urls to string "URL"
    - if the username isn't relevant then convert @username to "AT_USER"
    - converts emoji's to text description of the emoji
    """

In [ ]:
tweets_df = delete_duplicates(tweets_to_analyse)
tweets_df.reset_index(inplace=True, drop=True)
tweets_df['text_clean'] = tweets_df['text'].apply(preprocess_tweet)

## Tweets Analysis

Now it's time to analyse this data to try and get some interesting insights. Below are some questions you can answer to get started but the best Data Scientists are creative thinkers so think out of th ebox to decide what will be interesting insight

**What locations in Nigeria and around the world were the hashtags popular?**

**How far did the hashtag reach and how deep was this reach?**

**When the did movement start on social media? When did it reach critical mass?**

**What were the most popular words or terms used during the movement?**

**Who were the key influences and proponents involved in the movement?**

**What was the most popular tweet?**

**What is the general sentiment of the tweets about the movement? How has this changed from the start of the movement till now?**

**Were there any bots or bad actors spreading fake news around the movement?**

**What are some examples of this fake news?**

In [ ]:
tweet_fields=('author_id,context_annotations,created_at,entities,geo,id,public_metrics,possibly_sensitive,referenced_tweets,text'),
    user_fields=('id,location,name,public_metrics,username,verified'),
    place_fields=('country,full_name,geo,name,place_type'),
    expansions=('author_id,referenced_tweets.id,geo.place_id,referenced_tweets.id.author_id'),
    stringify=True)


## Sentiment Modelling
Train model on labelled twitter sentiment data from another dataset
<br>Next steps:
- Download Sentiment140 data
- preprocess tweet data
- Apply word embeddings to the text
- apply fasttext text classification
- Pickle model
- Apply model to endsars tweets text

### Get data ready for fastText training

Save as .txt file for the fastText classifer

In [ ]:
def prep_data(df, filename=""):
    """
    Create a functions that prepares the endsars data in a format that the fasttext model likes
    """

**Train Model**

**Apply Modelto Test data**

**Score Model**

**Check labels on example tweets**

### Apply trained fastText model to tweets df

In [ ]:
tweets_df['ft_predict'] = tweets_df.text.apply(ft_model.predict)
tweets_df["sentiment"] = tweets_df.ft_predict.apply(lambda x: 1 if x[0][0] == '__label__positive' else
                                                               0)
tweets_df['sentiment_prob'] = tweets_df.ft_predict.apply(lambda x: x[1][0])

In [ ]:
def print_tweet_sentiment(index=0):
    """
    Create function where you either pass the index of the tweet in the tweets_df or a tweet and it returns the 
    sentiment for the tweet
    """

In [ ]:
query = gen_request_parameters("EndSARS",results_per_call=10,start_time="2021-02-25",
    tweet_fields=('created_at,conversation_id,referenced_tweets,geo,geo.place_id,context_annotations.entity.id,context_annotations.entity.name,entities.hashtags,public_metrics,possibly_sensitive,text'),
    user_fields=('id,location,name,public_metrics,username,verified'),
    place_fields=('entities,country,full_name,geo,name,place_type'),
    expansions=('author_id,referenced_tweets.id,geo.place_id,referenced_tweets.id,referenced_tweets.id.author_id'),
    stringify=True)